CHARGEMENT DU MODELE

In [ ]:
from keras.models import load_model

# Charger le modèle U-Net sauvegardé
model = load_model('MODEL_VAC.h5')


CODE COUCHE SUPERIEURE DU MUSCLE

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# Paramètres de l'image (doivent correspondre à ceux utilisés pour l'entraînement)
img_height, img_width = 256, 256



# Chemin complet de l'image
image_path ='./UNET/IMG_SOURCE/PHOTO_K_600_REDIM/0594.png'
    
# Charger et prétraiter l'image de test
test_img = load_img(image_path, target_size=(img_height, img_width))
test_img_array = img_to_array(test_img) / 255.0  # Normalisation
test_img_array = np.expand_dims(test_img_array, axis=0)  # Ajouter une dimension pour le batch
    
# Prédire le masque pour l'image de test
predicted_mask = model.predict(test_img_array)
    
# Convertir la prédiction en masque binaire (0 ou 1)
predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
predicted_mask = np.squeeze(predicted_mask)  # Supprimer la dimension du batch
    
# Trouver les indices des pixels où l'objet est présent (valeurs de 1 dans le masque)
object_pixels = np.where(predicted_mask == 1)
    
if len(object_pixels[0]) > 0:  # Si des pixels de l'objet sont détectés
    # Récupérer les coordonnées xmin, xmax
    xmin = np.min(object_pixels[1])  # Le minimum des coordonnées x
    xmax = np.max(object_pixels[1])  # Le maximum des coordonnées x
    
    ymin = y_val = object_pixels[0][object_pixels[1] == xmin]  # Obtenir tous les y pour un x donné
    ymax = y_val2 = object_pixels[0][object_pixels[1] == xmax]  # Obtenir tous les y pour un x donné
    
    
    # Trouver les y min correspondant à chaque x entre xmin et xmax (limite inférieure)
    for x in range(xmin, xmax + 1):
        y_values = object_pixels[0][object_pixels[1] == x]  # Obtenir tous les y pour un x donné
        if len(y_values) > 0:  # S'assurer qu'il y a des pixels pour ce x
            y_min = np.min(y_values)  # Le y min pour ce x
            plt.scatter(x, y_min, color='green', s=0.1)  # Tracer les points verts pour le y min

    # Afficher les coordonnées
    print(f"Image : {image_path}, xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}")

    # Visualiser les points xmin et xmax sur l'image
    plt.imshow(np.squeeze(test_img_array))  # Afficher l'image originale en fond
    plt.scatter(xmin, np.min(object_pixels[0][object_pixels[1] == xmin]), color='red', label='xmin', s=10)  # Point rouge pour xmin
    plt.scatter(xmax, np.min(object_pixels[0][object_pixels[1] == xmax]), color='blue', label='xmax', s=10)  # Point bleu pour xmax
    plt.legend()
    plt.title(f'Image : {image_path} avec Limite Inférieure (xmin à xmax)')

else:
    print(f"Aucun objet détecté dans l'image {image_path}")




REFRAME IMAGE

In [3]:
def reframeImage(image, minX, maxX, minY, maxY):
    """
    Renvoie une portion de l'image.

    image: l'image à recadrer
    minX: Coordonnée X du coin supérieur gauche de l'image recadrée 
    maxX: Coordonnée X du coin inférieur droit de l'image recadrée
    minY: Coordonnée Y du coin supérieur gauche de l'image recadrée
    maxY: Coordonnée Y du coin inférieur droit de l'image recadrée

    return l'image recadrée
    """
    return image[minY:maxY, minX:maxX]

In [4]:
def measureFatThickness(image):
    """
    Mesure l'épaisseur minimale de la couche de gras sur une image.

    image: l'image à analyser

    return l'épaisseur de la couche de gras
    """
    # thresold d'acceptation
    differenceThresold = 70

    # Limites de la couche du gras
    topLimit = []
    bottomLimit = []

    # Recadrage manuel autour du nucleus medius
    image = reframeImage(image, xmin, xmax,0, 67)

    # Recherche des pixels qui délimitent la couche de gras
    for x in range(0, image.shape[1]):
        color = image[0][x][2]
        layers = 0
        for y in range(0, image.shape[0]):
            if layers > 1:
                break

            if (abs(int(image[y][x][2]) - color) > differenceThresold): 
                if layers == 0:
                    topLimit.append((x, y))
                    color = int(image[y][x][2])
                    image[y][x] = [0, 0, 255]

                else:
                    bottomLimit.append((x, y))
                    color = int(image[y][x][2])
                    image[y][x] = [0, 255, 0]

                layers += 1
    cv2.imshow('Epaisseur minimale de la couche de gras', image)    
    # Calcul de l'épaisseur minimale de la couche de gras
    distance, cords = findSmallestThickness(topLimit, bottomLimit)
    print("Epaisseur minimale de la couche de gras: " + str(distance) + " pixels")
    print("Points retenus : " + str(cords)+"\n")
    image[cords[0][1]][cords[0][0]] = [255, 0, 0]
    image[cords[1][1]][cords[1][0]] = [255, 0, 0]

    return distance,cords

In [5]:
def findSmallestThickness(topLimit, bottomLimit):
    """
    Trouve l'épaisseur minimale de la couche de gras.
    Calcule la distance minimale entre les pixels de la limite supérieure et inférieure de la couche de gras.

    topLimit: pixels de la limite supérieure de la couche de gras
    bottomLimit: pixels de la limite inférieure de la couche de gras

    return la distance minimale entre les deux ensembles de pixels
    """
    distance = 10000
    cords = []
    for pixelTop in topLimit:
        for pixelBottom in bottomLimit:
            d = math.sqrt((pixelTop[0] - pixelBottom[0])**2 + (pixelTop[1] - pixelBottom[1])**2)
            if d < distance:
                distance = d
                cords = [pixelTop, pixelBottom]

    return distance, cords

In [ ]:
fat_measure, coord_fat = measureFatThickness(cv2.imread(image_path))

In [6]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
import time
from sklearn.cluster import KMeans
from skimage.color import rgb2lab, lab2rgb, rgb2hsv, hsv2rgb
import math

In [ ]:
image =cv2.imread('./UNET/IMG_SOURCE/PHOTO_K_600_REDIM/0594.png')
# thresold d'acceptation
differenceThresold = 70

# Limites de la couche du gras
topLimit = []
bottomLimit = []

# Recadrage manuel autour du nucleus medius
image = reframeImage(image, xmin, xmax,0, 67)

# Recherche des pixels qui délimitent la couche de gras
for x in range(0, image.shape[1]):
    color = image[0][x][2]
    layers = 0
    for y in range(0, image.shape[0]):
        if layers > 1:
            break

        if (abs(int(image[y][x][2]) - color) > differenceThresold): 
            if layers == 0:
                topLimit.append((x, y))
                color = int(image[y][x][2])
                image[y][x] = [0, 0, 255]

            else:
                bottomLimit.append((x, y))
                color = int(image[y][x][2])
                image[y][x] = [0, 255, 0]

            layers += 1
cv2.namedWindow("Epaisseur minimale de la couche de gras", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Epaisseur minimale de la couche de gras",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
cv2.imshow("Epaisseur minimale de la couche de gras", image)    
# Calcul de l'épaisseur minimale de la couche de gras
distance, cords = findSmallestThickness(topLimit, bottomLimit)
print("Epaisseur minimale de la couche de gras: " + str(distance) + " pixels")
print("Points retenus : " + str(cords)+"\n")
image[cords[0][1]][cords[0][0]] = [255, 0, 0]
image[cords[1][1]][cords[1][0]] = [255, 0, 0]

print(distance,cords)